<a href="https://colab.research.google.com/github/Gjeffroy/LLM_finetuning/blob/main/Preparing_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets
! pip install pandas
! pip install openai

In [ ]:
# Replace with your actual API key
from google.colab import userdata

import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=userdata.get('openai_API_key'),
)


In [ ]:
from datasets import load_dataset

ds = load_dataset("louisbrulenaudet/code-urbanisme")

In [ ]:
ds['train']

Dataset({
    features: ['ref', 'title_main', 'texte', 'dateDebut', 'dateFin', 'num', 'id', 'cid', 'type', 'etat', 'nota', 'version_article', 'ordre', 'conditionDiffere', 'infosComplementaires', 'surtitre', 'nature', 'texteHtml', 'dateFinExtension', 'versionPrecedente', 'refInjection', 'idTexte', 'idTechInjection', 'origine', 'dateDebutExtension', 'idEliAlias', 'cidTexte', 'sectionParentId', 'multipleVersions', 'comporteLiensSP', 'sectionParentTitre', 'infosRestructurationBranche', 'idEli', 'sectionParentCid', 'numeroBo', 'infosRestructurationBrancheHtml', 'historique', 'infosComplementairesHtml', 'renvoi', 'fullSectionsTitre', 'notaHtml', 'inap'],
    num_rows: 2417
})

In [ ]:
import pandas as pd

df = pd.DataFrame(ds['train'])

In [ ]:
template = """Cite moi l'article {article}"""
df['article_id'] = df['ref'].str.split(',').str[-1].str.replace('art.', '')
df['instruction'] = df['article_id'].map(lambda x: template.format(article=x))
df[['article_id', 'instruction', 'texte']]

,article_id,instruction,texte
0,L101-1,Cite moi l'article L101-1,Le territoire français est le patrimoine commu...
1,L101-2,Cite moi l'article L101-2,Dans le respect des objectifs du développement...
2,L101-2-1,Cite moi l'article L101-2-1,L'atteinte des objectifs mentionnés au 6° bis ...
3,L101-3,Cite moi l'article L101-3,La réglementation de l'urbanisme régit l'utili...
4,L102-1,Cite moi l'article L102-1,L'autorité administrative compétente de l'Etat...
...,...,...,...
2412,A614-2,Cite moi l'article A614-2,Les architectes-conseils et les paysagistes-co...
2413,A614-3,Cite moi l'article A614-3,Les dépenses correspondants à la rémunération ...
2414,A614-4,Cite moi l'article A614-4,Les dispositions des articles A. 614-1 à A. 61...
2415,A431-10 Annexe,Cite moi l'article A431-10 Annexe,Attestation du contrôleur technique établissan...


In [ ]:


def generate_response(prompt):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-3.5-turbo",
  )
  return chat_completion.choices[0].message.content

#### Questions orientées (spécifique à l'article)

In [ ]:
prompt = """
Imagine une question dont la reponse se trouve dans le TEXTE suivant

TEXTE: \\n {}
"""

In [ ]:
test = prompt.format(df['texte'][0])
test

"\nImagine une question dont la reponse se trouve dans le TEXTE suivant\n\nTEXTE: \\n Le territoire français est le patrimoine commun de la nation. Les collectivités publiques en sont les gestionnaires et les garantes dans le cadre de leurs compétences. En vue de la réalisation des objectifs définis à l'article L. 101-2 , elles harmonisent leurs prévisions et leurs décisions d'utilisation de l'espace dans le respect réciproque de leur autonomie.\n"

In [ ]:
# test on the first 5 item of df
for i in range(5):
  print('TEXTE: ' + df['texte'][i])
  print(generate_response(prompt.format(df['texte'][i])))

TEXTE: Le territoire français est le patrimoine commun de la nation. Les collectivités publiques en sont les gestionnaires et les garantes dans le cadre de leurs compétences. En vue de la réalisation des objectifs définis à l'article L. 101-2 , elles harmonisent leurs prévisions et leurs décisions d'utilisation de l'espace dans le respect réciproque de leur autonomie.
Quels sont les organismes responsables de la gestion du territoire français ?
TEXTE: Dans le respect des objectifs du développement durable, l'action des collectivités publiques en matière d'urbanisme vise à atteindre les objectifs suivants : 1° L'équilibre entre : a) Les populations résidant dans les zones urbaines et rurales ; b) Le renouvellement urbain, le développement urbain et rural maîtrisé, la restructuration des espaces urbanisés, la revitalisation des centres urbains et ruraux, la lutte contre l'étalement urbain ; c) Une utilisation économe des espaces naturels, la préservation des espaces affectés aux activité

In [ ]:
# df['question_orientée'] = df['texte'].map(lambda x: generate_response(prompt.format(x)))

In [ ]:
df[['article_id', 'question_orientée', 'texte']]

,article_id,question_orientée,texte
0,L101-1,Quels acteurs sont désignés comme gestionnaire...,Le territoire français est le patrimoine commu...
1,L101-2,Quels sont les objectifs du développement dura...,Dans le respect des objectifs du développement...
2,L101-2-1,Quels sont les critères à prendre en compte po...,L'atteinte des objectifs mentionnés au 6° bis ...
3,L101-3,Quels sont les territoires exclus de la réglem...,La réglementation de l'urbanisme régit l'utili...
4,L102-1,Quelles sont les deux conditions requises pour...,L'autorité administrative compétente de l'Etat...
...,...,...,...
2412,A614-2,Quel est le montant de la rémunération au titr...,Les architectes-conseils et les paysagistes-co...
2413,A614-3,Quels sont les crédits sur lesquels sont imput...,Les dépenses correspondants à la rémunération ...
2414,A614-4,Quelles sont les autorités compétentes pour mo...,Les dispositions des articles A. 614-1 à A. 61...
2415,A431-10 Annexe,Qu'est-ce que le contrôleur technique atteste ...,Attestation du contrôleur technique établissan...


In [ ]:
# save df
df.to_csv('df_questions_orientees.csv', index=False)

#### Réponses orientés

In [1]:
import pandas as pd
# load df
df = pd.read_csv('df_questions_orientees.csv')

In [2]:
prompt = """
Repond à la QUESTION en utilisant le TEXTE suivant. La réponse doit être factuelle et uniquement basée sur le TEXTE.

QUESTION:\\n {} \\n
TEXTE: \\n {}

Renvoi juste la reponse sans la question ni le texte.
"""

In [3]:
from openai import AsyncOpenAI
import openai
import backoff

import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_API_key')

client = AsyncOpenAI()


# Limit to 500 requests per minute
RATE_LIMIT = 500
TIME_FRAME = 60  # seconds

async def generate_response(prompt):
  messages = [{"role": "user", "content": prompt}]

  response = await client.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=messages,
              timeout=1000
          )
  return response.choices[0].message.content

# Define an async function with rate limiting
async def generate_response_limited(prompt, semaphore):
    async with semaphore:  # Only allow a set number of tasks at a time
        response = await generate_response(prompt)
        return response

In [11]:
async def main():
    # Prepare prompts
    prompts = [prompt.format(df['question_orientée'][i], df['texte'][i]) for i in range(len(df))]

    # Semaphore to limit concurrent tasks
    semaphore = asyncio.Semaphore(RATE_LIMIT)

    # Run tasks with a rate limit and tqdm for progress
    results = []
    tasks = []
    for i, prompt in enumerate(prompts):
        if i % RATE_LIMIT == 0 and i != 0:  # Check if we need to wait to respect the rate limit
            await asyncio.sleep(TIME_FRAME)

        # Append task with the semaphore
        tasks.append(generate_response_limited(prompt, semaphore))

    # Use tqdm to track progress
    for task in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
        try :
          results.append(await task)
        except Exception as e:
          print(e)
          results.append('')

    return results

# Run the main asynchronous function
results = await main()

# Store results in DataFrame if needed
df['reponse_orientée'] = results

NameError: free variable 'prompt' referenced before assignment in enclosing scope

In [ ]:
import asyncio
from tqdm.notebook import tqdm

async def main():
    # Prepare prompts for async API calls
    prompts = [prompt.format(df['question_orientée'][i], df['texte'][i]) for i in range(len(df))]

    # Create tasks for each prompt
    tasks = [generate_response(p) for p in prompts]

    # Use tqdm to show progress as each task completes
    results = []
    for task in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
        try :
          results.append(await task)
        except Exception as e:
          print(e)
          results.append('')

    return results

# Run the main asynchronous function
results = await main()

# Store results in DataFrame if needed
df['reponse_orientée'] = results

  0%|          | 0/2417 [00:00<?, ?it/s]

In [ ]:
from tqdm.notebook import tqdm
df['reponse_orientée'] = ''
for i in tqdm(range(len(df))):
  # print progress on line 1
  print(i, end='\r')
  prompt_full = prompt.format(df['question_orientée'][i],df['texte'][i])
  df.loc[i, 'reponse_orientée'] = generate_response(prompt_full)

/usr/local/lib/python3.10/dist-packages/traitlets/traitlets.py:264: RuntimeWarning: coroutine 'AsyncCompletions.create' was never awaited
  self.value = value


  0%|          | 0/2417 [00:00<?, ?it/s]

AttributeError: 'coroutine' object has no attribute 'choices'

In [ ]:
import asyncio
from tqdm.notebook import tqdm

# Define an async wrapper for generate_response
async def async_generate_response(prompt_full):
    return await generate_response(prompt_full)  # Ensure generate_response is async or adjust accordingly

# Main function to process all rows asynchronously
async def process_responses(df):
    tasks = []
    for i in range(len(df)):
      prompt_full = prompt.format(df['question_orientée'][i], df['texte'][i])
      tasks.append(async_generate_response(prompt_full))

    # Run all tasks with tqdm to track progress
    responses = []
    for response in asyncio.as_completed(tasks):
      print(response, end='\r')
      responses.append(await response)

    # Update the DataFrame with results
    df['reponse_orientée'] = responses

# To run the async function
await process_responses(df)


  0%|          | 0/2417 [00:00<?, ?it/s]